<a href="https://colab.research.google.com/github/OlgaSeleznova/ML_toolbox/blob/main/Lyrics_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup modules

In [5]:
! git clone https://github.com/OlgaSeleznova/ML_toolbox.git

Cloning into 'ML_toolbox'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 90 (delta 40), reused 14 (delta 4), pack-reused 0
Unpacking objects: 100% (90/90), done.


In [13]:
#install modules
import sys
sys.path.insert(0,'/ML_toolbox/NLP_preprocessing')

# ! pip install langid
# ! pip install NLP_preprocessing

In [2]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import re
from ML_toolbox.NLP_preprocessing import English_preprocess   #custom module for English preprocessing
import langid   # package to identify language of lyrics


In [3]:
# load data
from google.colab import drive
drive.mount('/content/drive/')

lyrics = pd.read_parquet('/content/drive/My Drive/ML_toolbox/data/metrolyrics.parquet')
lyrics.shape

Mounted at /content/drive/


(49976, 8)

In [4]:
# identify language of the lyric
lyrics['language'] = lyrics['lyrics'].apply(lambda x: langid.classify(x)[0])
lyrics['language'].value_counts()[:10]

en    46332
es     1381
de      668
fr      277
it      269
pt      123
sw      118
fi       84
no       84
sv       53
Name: language, dtype: int64

Since English posts are 46k out of 49k posts total, and text classification for multiple languages should be prepared separately, we will strat with only English.

In [5]:
lyric_en = lyrics[lyrics['language'] == 'en'].copy()
lyric_en.shape

(46332, 9)

Use English preprocessing class from this repository to clean data

In [6]:
# initialize an object from custom class for English preprocessing
cleaner = English_preprocess.English_preprocessing()
# create a list of tuples to add custom cleaning with regex
to_replace = [(r'[^a-zA-Z]',' '), (r'(\s+)',' ')]
# clean posts with regex
lyric_en['lyrics_cleaned'] = lyric_en['lyrics'].apply(lambda x: cleaner.regex_cleaner(x,to_replace))
# tokenize lyrics, remove stopwords and join to string.  
lyric_en['lyrics_cleaned'] = lyric_en['lyrics_cleaned'].apply(lambda x: ' '.join(cleaner.tokenize(x)))

In [7]:
#encode genre names
genre_srt_to_int = {'Rock':0, 'Pop':1, 'Hip-Hop':2, 'Metal':3, 'Country':4}
lyric_en['genre'] = lyric_en['genre'].replace(genre_srt_to_int)

# rename columns to more suitable
lyric_en = lyric_en.rename(columns={'lyrics_cleaned':'text','genre':'label'})

Now posts are cleaned and we can divide data into train, validation and test sets.

In [8]:
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set
x_train, x_test, y_train, y_test = train_test_split(lyric_en['text'], lyric_en['label'], test_size=1 - train_ratio, random_state=42)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state=42) 

print(x_train.shape, x_val.shape, x_test.shape)

(34749,) (6949,) (4634,)


Save datasets to the folder

In [11]:
def create_datasets(x, y, file_name):
    df = pd.concat([x, y], axis=1, ignore_index=False, sort=False)
    print('Dataset shape: ',df.shape)
    df.to_csv('/content/drive/My Drive/ML_toolbox/data/eng_lyrics/' + file_name, index=False)

In [10]:
! mkdir data
! mkdir data/eng_lyrics/

In [12]:
create_datasets(x_train, y_train, 'train_t.csv')
create_datasets(x_val, y_val, 'valid_t.csv')
create_datasets(x_test, y_test, 'test_t.csv')

Dataset shape:  (34749, 2)
Dataset shape:  (6949, 2)
Dataset shape:  (4634, 2)
